# 什么是梯度

像平时学习的倒数，微分方程，其实都是对x轴或者是y轴求导，求曲线沿着某个轴线的变化速率。

例如y=1/x .求导之后可以发现，当x越大，求导值越小，说明曲线的变化速率越小

什么是梯度呢？梯度是一个矢量，并不局限于x或者y轴。而是x轴和y轴方向的一个矢量和。梯度反应的是函数变化速率最快的方向。矢量的模越大，则函数在这个方向变化越快

# 怎么搜索最优解

最优解一般包括全局最优解和局部最优解。局部最优解一般很容易找到，全局最优解往往会遇到很多问题

在二维图像上，搜索最优解。先随机初始化一个x。求出该点的斜率，用一个学习率乘以该斜率。x加表示求全局最大值。x减表示求全局最小值(因为梯度是函数增长最快的方向)

在高维图形中，也是先初始化值。然后求出梯度，接下来和上面一样。x-func(x,y)偏x，y-func(x,y)偏y

# 什么会影响你的搜索

- 初始化的值会影响搜索最优解。初始化的值不同，可能会取得不同的最优解
- 学习率也会影响。学习率越大，可能效果越差。迭代次数会变多。甚至可能不收敛。

# 激活函数

神经元有多个输入，将这些输入求和加上偏置bias后，作为激活函数的输入，然后得到一个输出。这就是激活函数的作用。

第一个激活函数是根据动物得到的。在刺激动物的时候，神经元不会马上作出反应，而是达到一定阈值才会。所以提出使用阶梯函数作为激活函数。但是阶梯函数求导为0，因此不适合优化，所以提出了sigmoid函数

In [1]:
import torch

In [2]:
a = torch.linspace(-100,100,10)
a

tensor([-100.0000,  -77.7778,  -55.5556,  -33.3333,  -11.1111,   11.1111,
          33.3333,   55.5556,   77.7778,  100.0000])

In [3]:
torch.sigmoid(a)

tensor([0.0000e+00, 1.6655e-34, 7.4564e-25, 3.3382e-15, 1.4945e-05, 9.9999e-01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])

In [4]:
# tanh
torch.tanh(a)

tensor([-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.])

In [6]:
# relu又叫合页函数
# relu大于0的时候梯度不变保持为1,因此容易出现梯度爆炸
# 优先使用这个函数作为深度学习的激活函数

torch.relu(a)

tensor([  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,  11.1111,  33.3333,
         55.5556,  77.7778, 100.0000])

# loss

## 使用pytorch自动求导

损失函数表现为预测值和实际值的差值平方和再求平均，mean suqure loss

为了使损失函数最小，这个问题是一个凸优化问题。也涉及到求梯度等方法

现在假设一个例子，实现求loss

y = wx,y and x是知道的，我们初始化一个w

mean_squre_loss = (y-wx)^2是一个关于w的函数 =(1-w)^2 我们知道loss最小w应该取1

对w求导后,2(y-wx)*-x

假设y=1,x=1,初始化w=2 ,2(1-w)*-1 = 2(w-1)

假设w初始化为2，那么求导后结果为2

In [7]:
x = torch.ones(1)
x # dim=1

tensor([1.])

In [16]:
predict = torch.ones(1)

In [11]:
w = torch.full([1],2.)
w

tensor([2.])

In [12]:
from torch.nn import functional as F

In [17]:
mes = F.mse_loss(predict, w*x) #第一个系数是预测值，第二个参数是表达式，y=后面这部分的/也是准确值标签，和预测值相对

In [19]:
mes # 这里预测值是1，实际值是2，那么loss就是1

tensor(1.)

In [20]:
loss = (predict-x*w)**2

In [22]:
# 会报错是因为没有设置跟踪w求导
# pytorch是动态图，在设置的时候，需要手动设置哪些系数需要求导
torch.autograd.grad(loss, w) # 第一个是方程，第二个是需要求导的系数

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [23]:
# 我们在这里设置追踪自动求导，因为后面要对w求导，也可以后面手动补全
w.requires_grad_() 
# w = torch.full([1],2.,requires_grad_=True),也可以在创建的时候直接设置

tensor([2.], requires_grad=True)

In [24]:
torch.autograd.grad(loss, w)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [25]:
loss = (predict-x*w)**2 # 必须要继续更新才能求导

In [26]:
torch.autograd.grad(loss, w)

(tensor([2.]),)

## 使用backward求梯度

除了使用torch.autograd计算梯度以外，还可以使用backward计算梯度

请看https://www.cnblogs.com/marsggbo/p/11549631.html

In [31]:
# 前面我们求了损失函数

mes = F.mse_loss(predict, w*x) #第一个系数是预测值，第二个参数是表达式，y=后面这部分的/也是准确值标签，和预测值相对

In [32]:
mes # mes 是一个floattensor类型。我们可以使用这个类型的backward计算查看w的梯度

tensor(1., grad_fn=<MeanBackward0>)

In [33]:
mes.backward()

In [35]:
w.grad # 查看此时的梯度

tensor([2.])

## 总结两种方式求梯度

**总结：两种方式求梯度**

- torch.autograd.grad(loss, [w1,w2...])

- loss.backward()

# softmax

https://blog.csdn.net/bitcarmanlee/article/details/82320853

该元素的softmax值，就是该元素的指数与所有元素指数和的比值。所以数值大的会被放大。

In [39]:
a = torch.rand(3) # 在建立变量的时候应该手动设置是否追踪求导
a

tensor([0.5787, 0.7905, 0.3651])

In [40]:
a.requires_grad_()

tensor([0.5787, 0.7905, 0.3651], requires_grad=True)

In [42]:
p = F.softmax(a, dim=0)
p

tensor([0.3286, 0.4061, 0.2654], grad_fn=<SoftmaxBackward>)